In [ ]:
import math
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

In [ ]:
def features(x, p=10):
    phi = torch.zeros(x.numel(), p)
    ind = 0
    for freq in range(p//2):
        phi[:, ind] = torch.cos((freq+1)*math.pi*x)
        ind += 1
        phi[:, ind] = torch.sin((freq+1)*math.pi*x)
        ind += 1
            
    return phi

In [ ]:
def response(phi, sig=1.):
    alpha = torch.randn(phi.size(-1))
    y = phi.matmul(alpha)
    y = y + torch.randn(y.shape)*sig
    return y

In [ ]:
def post_cov(phi, sig, s_0):
    in_ = phi.t().matmul(phi).div(sig) + s_0.inverse()
#     in_ = phi.t().matmul(phi).div(sig)
    return in_.inverse()

In [ ]:
def func_cov(phi, sig, s_0):
    in_ = phi.t().matmul(phi).div(sig) + s_0.inverse()
#     in_ = phi.t().matmul(phi).div(sig)
    return phi.matmul(in_.inverse()).matmul(phi.t())

## Setup Data

In [ ]:
nx = 300
P = 200

x = torch.linspace(0, 50, nx)
phi = features(x, p=P)
sig = 0.1
y = response(phi, sig)
alpha = 3.

In [ ]:
prior_cov = alpha * torch.eye(P)

In [ ]:
plt.plot(y)

### Begin Modeling

In [ ]:
gram = phi.t().matmul(phi)
eigs, evecs = np.linalg.eig(gram)
eigs = np.real(eigs)

In [ ]:
order_to_add = np.random.permutation(nx)

In [ ]:
keepers = []
eigs = torch.zeros(nx, P)
func_eigs = []

for ii, idx in enumerate(order_to_add):
    keepers.append(idx)
    phi_temp = phi[keepers,:]
    
    posterior_covariance = post_cov(phi_temp, sig, prior_cov)
    evals, _ = np.linalg.eig(posterior_covariance)
    eigs[ii, :] = torch.FloatTensor(np.real(evals))
    
    function_covariance = func_cov(phi_temp, sig, prior_cov)
    evals, _ = np.linalg.eig(function_covariance)
    func_eigs.append(torch.FloatTensor(np.real(evals)))

In [ ]:
def effective_dimension(eigs, alpha=1.):
    nx = eigs.size(0)
    eff_dim = torch.zeros(nx)
    for ii in range(nx):
        eff_dim[ii] = torch.sum(eigs[ii, :].div(eigs[ii, :] + alpha))
        
    return eff_dim

In [ ]:
alpha = 5.
eff_dim = effective_dimension(eigs, alpha=alpha)
func_eff_dim = [torch.sum(ee.div(ee + alpha)) for ee in func_eigs]

In [ ]:
import seaborn as sns

In [ ]:
title_fs = 28
ax_fs = 16
leg_fs = 14
tick_size = 16

sns.set_style("white")
colors = sns.color_palette("bright")
# colors = ["537EC5", "F39422"]

fig, ax1 = plt.subplots(figsize=(6, 5))
l1, = ax1.plot(eff_dim, color=colors[0], label="Eff. Dim. Posterior Covariance")
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)
ax1.grid()

ax2 = ax1.twinx()
l2, = ax2.plot(func_eff_dim, color=colors[1], label="Eff. Dim. Function")

# l3 = ax1.axvline(200, color='k', label="Number of Parameters", linestyle=":")
plt.title("Bayesian Linear Regression", fontsize=title_fs)
ax1.set_xlabel("Number of Data Points", fontsize=ax_fs)
ax1.set_ylabel("Effective Dimension (Parameter Space)", fontsize=ax_fs)
ax2.set_ylabel("Effective Dimension (Function Space)", fontsize=ax_fs)

plt.xlim(0, nx)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

ax1.axvspan(0, 175, alpha=0.1, color='red')
ax1.axvspan(175, 225, alpha=0.1, color='yellow')
ax1.axvspan(225, nx, alpha=0.1, color='green')

lines = [l1, l2]
plt.legend(lines, [l.get_label() for l in lines], loc="center right",
          fontsize=leg_fs)

# plt.legend(fontsize=leg_fs)
plt.savefig("./blr_eff_dim.pdf", bbox_inches="tight")


## Load in NN Data

In [ ]:
import pickle

In [ ]:
def compute_eff_dim(x, s = 1.):
    return np.sum(x / (x + s))

In [ ]:
fpath = "../saved-experiments/"
fname = "bnn_regression_evals.pkl"
file = open(fpath + fname, 'rb')
all_evals = pickle.load(file)
file.close()

In [ ]:
all_evals['eigs'][0][0][1]
all_n = all_evals['n']

In [ ]:
eff_dims_parameters = [[compute_eff_dim(all_evals['eigs'][i][x][0], s=1.) for x in range(len(all_n))] for i in range(len(all_evals['eigs']))]
eff_dims_fns = [[compute_eff_dim(all_evals['eigs'][i][x][1], s=1e-10) for x in range(len(all_n))] for i in range(len(all_evals['eigs']))]

In [ ]:
par_eff_dim = np.array(eff_dims_parameters)
par_eff_dim = np.mean(par_eff_dim, 0)

fn_eff_dim = np.array(eff_dims_fns)
fn_eff_dim = np.mean(fn_eff_dim, 0)

In [ ]:
title_fs = 28
ax_fs = 26
leg_fs = 26
tick_size = 16

sns.set_style("white")
colors = sns.color_palette("bright")

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

l1, = ax1.plot(eff_dim, color=colors[0], label="Eff. Dim. Posterior Covariance",
              linewidth=3.)
plt.xticks(fontsize=tick_size)
plt.yticks(fontsize=tick_size)

ax12 = ax1.twinx()
l2, = ax12.plot(func_eff_dim, color=colors[1], label="Eff. Dim. Function",
               linewidth=4., linestyle="--")

plt.title("Bayesian Linear Regression", fontsize=title_fs)
ax1.set_xlabel("Number of Data Points", fontsize=ax_fs)
ax1.set_ylabel(r"$N_{eff}$ (Parameter Cov.)", fontsize=ax_fs)

ymin = 0
ymax = 80
ax1.set_xlim(0, nx)
ax1.set_ylim(ymin, ymax)
ax1.tick_params(axis='both', labelsize=tick_size)
ax12.tick_params(axis='y', labelsize=tick_size)

ax1.axvspan(0, 175, alpha=0.05, color='red')
ax1.axvspan(175, 225, alpha=0.1, color='yellow')
ax1.axvspan(225, nx, alpha=0.2, color='green')
ax1.axvline(225, alpha=0.5, color='k')
ax1.axvline(175, alpha=0.5, color='k')



ax2.scatter(all_n, par_eff_dim, color=colors[0], label="Eff. Dim. Posterior Covariance",
           s=75)

ax22 = ax2.twinx()
ax22.scatter(all_n, fn_eff_dim, color=colors[1], label="Eff. Dim. Function", s=75,
            marker='D')

plt.title("Bayesian Neural Network", fontsize=title_fs)
ax2.set_xlabel("Number of Data Points", fontsize=ax_fs)
ax22.set_ylabel(r"$N_{eff}$ (Hessian)", fontsize=ax_fs)
ax2.tick_params(axis='both', labelsize=tick_size)
ax22.tick_params(axis='y', labelsize=tick_size)

xmin = 0
xmax = all_n[-1]
ymin = 0
ymax = 200

ax2.set_xlim(xmin, xmax)
ax2.set_ylim(ymin, ymax)
ax2.axvspan(0, 500, alpha=0.05, color='red')
ax2.axvspan(500, 750, alpha=0.1, color='yellow')
ax2.axvspan(750, all_n[-1], alpha=0.2, color='green')
ax2.axvline(500, alpha=0.5, color='k')
ax2.axvline(750, alpha=0.5, color='k')

fig.subplots_adjust(wspace=0.2, hspace=None)

ax1.legend(handles = [l1,l2] , labels=["Parameter Covariance", "Hessian"],loc='upper center', 
             bbox_to_anchor=(1., -0.12),fancybox=False, shadow=False, ncol=3,
            fontsize=leg_fs)
plt.savefig("./bayes-eff-dim.pdf", bbox_inches="tight")
plt.show()
